In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('date_time').getOrCreate()

In [6]:
#CSV Required to infer schema and header so it is readable for us
df = spark.read.csv('input_data/appl_stock.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [7]:
df.head(1)

[Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [9]:
df.select(["Date", "Open"]).show(5)

+----------+----------+
|      Date|      Open|
+----------+----------+
|2010-01-04|213.429998|
|2010-01-05|214.599998|
|2010-01-06|214.379993|
|2010-01-07|    211.75|
|2010-01-08|210.299994|
+----------+----------+
only showing top 5 rows



In [10]:
# functions
from pyspark.sql.functions import (dayofmonth, hour, dayofyear, month,
                                   year, weekofyear, format_number, date_format)

In [13]:
#using select
df.select(dayofmonth('Date')).show(2)
df.select(hour('Date')).show(2)
df.select(year('Date')).show(2)

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
+----------------+
only showing top 2 rows

+----------+
|hour(Date)|
+----------+
|         0|
|         0|
+----------+
only showing top 2 rows

+----------+
|year(Date)|
+----------+
|      2010|
|      2010|
+----------+
only showing top 2 rows



In [26]:
#grab average close price by year
closing_year = df.select([year('Date').alias('year'), 'Close'])
closing_year.groupBy('year').mean().show()

+----+---------+------------------+
|year|avg(year)|        avg(Close)|
+----+---------+------------------+
|2015|   2015.0|120.03999980555547|
|2013|   2013.0| 472.6348802857143|
|2014|   2014.0| 295.4023416507935|
|2012|   2012.0| 576.0497195640002|
|2016|   2016.0|104.60400786904763|
|2010|   2010.0| 259.8424600000002|
|2011|   2011.0|364.00432532142867|
+----+---------+------------------+



In [36]:
#create new a new column 
newdf = df.withColumn("Year", year(df['Date']))
result = newdf.groupBy("Year").mean().select(['Year', format_number('avg(Close)', 2).alias('Average Closing')]).show()

+----+---------------+
|Year|Average Closing|
+----+---------------+
|2015|         120.04|
|2013|         472.63|
|2014|         295.40|
|2012|         576.05|
|2016|         104.60|
|2010|         259.84|
|2011|         364.00|
+----+---------------+

